<a href="https://colab.research.google.com/github/Sylwia0506/Data_Analisys-Covid19/blob/main/Data_Analisys_Covid19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Covid19

In [ ]:
!pip install beautifulsoup4

In [63]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup as s
from urllib.request import Request, urlopen
from datetime import date, datetime
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

today = datetime.now()
yesterday = "%s %d, %d" %(date.today().strftime("%b"), today.day-1, today.year)
#print(yesterday)

url = "https://www.worldometers.info/coronavirus/#countries"
request = Request(url, headers={'User-Agent': "Google"})

webpage = urlopen (request)
#print(webpage)

page = s(webpage, "html.parser")

table = page.findAll("table",{"id" : "main_table_countries_yesterday"} )
#print(table)

c = table[0].findAll("tr",{"style":""})

title = c[0]
del c[0]

all_data = []
clean = True
for country in c:
  country_data = []
  country_c = country.findAll("td")

  if country_c[1].text == "Poland":
      continue
  for i in range(1, len(country_c)):
    final_feature = country_c[i].text
    if clean:
      if i!=1 and i!= len(country_c)-1:
        final_feature = final_feature.replace(",","")
        if final_feature.find('+') != -1:
          final_feature = final_feature.replace("+","")
          final_feature = float(final_feature)
        elif final_feature.find('-') != -1:
          final_feature = final_feature.replace("-","")
          final_feature = float(final_feature)*-1
  
    if final_feature == 'N/A':
      final_feature = 0
    elif final_feature == "" or final_feature == " ":
      final_feature = -1

    country_data.append(final_feature)
  all_data.append(country_data)
    
df = pd.DataFrame(all_data)
df = df.drop([15,16,17,18,19,20], axis = 1 )
name_column = ["Country", "Total Cases", "New Cases", "Total Deaths", "New Deaths", "Total Recovered", "New Recovered",
               "Serious/Critical", "Active Cases", "Total Cases/1M", "Deaths/1M", "Total Tests","Test/1M", "Population", "Continent"]
   
df.columns = name_column

df.head()


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Serious/Critical,Active Cases,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent
0,World,192229746,512049.0,4133350,8292.0,174924706,370049.0,13171690,81822,24661,530.3,-1,-1,-1,All
1,China,92342,65.0,4636,-1.0,87098,22.0,608,14,64,3,160000000,111163,1439323776,Asia
2,USA,35082489,45002.0,625373,266.0,29435171,23215.0,5021945,6349,105340,1878,518658231,1557344,333040210,North America
3,India,31215142,42123.0,418511,489.0,30383001,36870.0,413630,8944,22389,300,447341133,320851,1394235023,Asia
4,Brazil,19419741,27896.0,544302,1425.0,18124621,57541.0,750818,8318,90685,2542,54786381,255838,214145148,South America


In [66]:
for i in df.columns:
  if i != 'Country' and i != 'Continent':
    df[i] = pd.to_numeric(df[i])

df["%Inc Cases"] =df["New Cases"]/df["Total Cases"]*100
df["%Inc Deaths"] =df["New Deaths"]/df["Total Deaths"]*100
df["%Inc Recovered"] =df["New Recovered"]/df["Total Recovered"]*100

df.head()


,Country,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered,Serious/Critical,Active Cases,Total Cases/1M,Deaths/1M,Total Tests,Test/1M,Population,Continent,%Inc Cases,%Inc Deaths,%Inc Recovered
0,World,192229746,512049.0,4133350,8292.0,174924706,370049.0,13171690,81822,24661.0,530.3,-1,-1,-1,All,0.266373,0.200612,0.211548
1,China,92342,65.0,4636,-1.0,87098,22.0,608,14,64.0,3.0,160000000,111163,1439323776,Asia,0.070391,-0.021570,0.025259
2,USA,35082489,45002.0,625373,266.0,29435171,23215.0,5021945,6349,105340.0,1878.0,518658231,1557344,333040210,North America,0.128275,0.042535,0.078868
3,India,31215142,42123.0,418511,489.0,30383001,36870.0,413630,8944,22389.0,300.0,447341133,320851,1394235023,Asia,0.134944,0.116843,0.121351
4,Brazil,19419741,27896.0,544302,1425.0,18124621,57541.0,750818,8318,90685.0,2542.0,54786381,255838,214145148,South America,0.143648,0.261803,0.317474


In [79]:
df.agg(
    {
    "Total Cases" : ["min", "max", "median"],
    "New Cases" : ["min", "max", "median"],
    "Total Deaths" : ["min", "max","median"],
    "New Deaths" : ["min", "max","median"],
    "Total Recovered": ["min", "max","median"],
    "New Recovered": ["min", "max","median"],
    }
)

,Total Cases,New Cases,Total Deaths,New Deaths,Total Recovered,New Recovered
min,10.0,-1.0,-1.0,-1.0,0.0,-1.0
max,192229746.0,512049.0,4133350.0,8292.0,174924706.0,370049.0
median,90910.0,155.0,1236.0,1.0,73340.0,43.0
